# Lab 3: Writing Compositional Code with Generative Stubs

In classical programming, pure (stateless) functions are a simple and powerful abstraction. A pure function takes inputs, computes outputs, and has no side effects. Generative programs can also use functions as abstraction boundaries, but in a generative program the meaning of the function can be given by an LLM instead of an interpreter or compiler. This is the idea behind a GenerativeSlot.

A GenerativeSlot is a function whose implementation is provided by an LLM. In Mellea, you define these using the `@generative` decorator. The function signature specifies the interface, and the docstring (or type annotations) guides the LLM in producing the output.

In this lab, we will see how **compositionality checks** can be used to combine libraries of generative functions.

In [ ]:
# Import necessary libraries
import mellea
from mellea import generative

# Type hinting
from typing import Literal

# Display utilities
from IPython.display import display, Markdown

# Format code cells with black
import jupyter_black

jupyter_black.load()

**`@generative`**: Converts a function into an AI-powered function.

This decorator transforms a regular Python function into one that uses an LLM to generate outputs. The function's entire signature - including its name, parameters, docstring, and type hints - is used to instruct the LLM to imitate that function's behavior. The output is guaranteed to match the return type annotation using structured outputs and automatic validation.

**Note:** 
> Works with async functions as well.

**Note:** 
> Write the function and docstring in the most Pythonic way possible, not like a prompt. This ensures the function is well-documented easily understood, and familiar to any Python developer. The more natural and conventional your function definition, the better the AI will understand and imitate it.

In [ ]:
################################################################################
# SUMMARIZER LIBRARY                                                           #
################################################################################


@generative
def summarize_meeting(transcript: str) -> str:
    """Summarize the meeting transcript into a concise paragraph of main points."""


@generative
def summarize_contract(contract_text: str) -> str:
    """Produce a natural language summary of contract obligations and risks."""


@generative
def summarize_short_story(story: str) -> str:
    """Summarize a short story, with one paragraph on plot and one paragraph on broad themes."""


################################################################################
# DECISION AIDES LIBRARY                                                       #
################################################################################


# The Decision Aides Library
@generative
def propose_business_decision(summary: str) -> str:
    """Given a structured summary with clear recommendations, propose a business decision."""


@generative
def generate_risk_mitigation(summary: str) -> str:
    """If the summary contains risk elements, propose mitigation strategies."""


@generative
def generate_novel_recommendations(summary: str) -> str:
    """Provide a list of novel recommendations that are similar in plot or theme to the short story summary."""

## Summarizing a meeting

Let's use the meeting summarizer to summarize a meeting transcript.

In [ ]:
################################################################################
# Transcript of a meeting discussing risks                                     #
################################################################################

transcript = """Meeting Transcript: Market Risk Review -- Self-Sealing Stembolts Division
Date: December 1, 3125
Attendees:

Karen Rojas, VP of Product Strategy

Derek Madsen, Director of Global Procurement

Felicia Zheng, Head of Market Research

Tom Vega, CFO

Luis Tran, Engineering Liaison

Karen Rojas:
Thanks, everyone, for making time on short notice. As you've all seen, we've got three converging market risks we need to address: tariffs on micro-carburetors, increased adoption of the self-interlocking leafscrew, and, believe it or not, the "hipsterfication" of the construction industry. I need all on deck and let's not waste time. Derek, start.

Derek Madsen:
Right. As of Monday, the 25% tariff on micro-carburetors sourced from the Pan-Alpha Centauri confederacy is active. We tried to pre-purchase a three-month buffer, but after that, our unit cost rises by $1.72. That's a 9% increase in the BOM cost of our core model 440 stembolt. Unless we find alternative suppliers or pass on the cost, we're eating into our already narrow margin.

Tom Vega:
We cannot absorb that without consequences. If we pass the cost downstream, we risk losing key mid-tier OEM clients. And with the market already sniffing around leafscrew alternatives, this makes us more vulnerable.

Karen:
Lets pause there. Felicia, give us the quick-and-dirty on the leafscrew.

Felicia Zheng:
It's ugly. Sales of the self-interlocking leafscrew—particularly in modular and prefab construction—are up 38% year-over-year. It's not quite a full substitute for our self-sealing stembolts, but they are close enough in function that some contractors are making the switch. Their appeal? No micro-carburetors, lower unit complexity, and easier training for install crews. We estimate we've lost about 12% of our industrial segment to the switch in the last two quarters.

Karen:
Engineering, Luis; your take on how real that risk is?

Luis Tran:
Technically, leafscrews are not as robust under high-vibration loads. But here's the thing: most of the modular prefab sites don not need that level of tolerance. If the design spec calls for durability over 10 years, we win. But for projects looking to move fast and hit 5-year lifespans? The leafscrew wins on simplicity and cost.

Tom:
So they're eating into our low-end. That's our volume base.

Karen:
Exactly. Now let's talk about this last one: the “hipsterfication” of construction. Felicia?

Felicia:
So this is wild. We're seeing a cultural shift in boutique and residential construction—especially in markets like Beckley, West Sullivan, parts of Osborne County, where clients are requesting "authentic" manual fasteners. They want hand-sealed bolts, visible threads, even mismatched patinas. It's an aesthetic thing. Function is almost secondary. Our old manual-seal line from the 3180s? People are hunting them down on auction sites.

Tom:
Well, I'm glad I don't have to live in the big cities... nothing like this would ever happen in downt-to-earth places Brooklyn, Portland, or Austin.

Luis:
We literally got a request from a design-build firm in Keough asking if we had any bolts “pre-distressed.”

Karen:
Can we spin this?

Tom:
If we keep our vintage tooling and market it right, maybe. But that's niche. It won't offset losses in industrial and prefab.

Karen:
Not yet. But we may need to reframe it as a prestige line—low volume, high margin. Okay, action items. Derek, map alternative micro-carburetor sources. Felicia, get me a forecast on leafscrew erosion by sector. Luis, feasibility of reviving manual seal production. Tom, let's scenario-plan cost pass-through vs. feature-based differentiation.

Let's reconvene next week with hard numbers. Thanks, all."""

In [ ]:
# Start a Mellea session
m = mellea.start_session()

# Summarize the meeting
summary = summarize_meeting(m, transcript=transcript)

# Display the summary
display(Markdown(f"""# Summary of meeting:\n {summary}"""))

## Sentiment Analysis

Let's create a simple sentiment analysis tool.

In [ ]:
@generative
def sentiment_classifier(text: str) -> Literal["positive", "negative"]:
    """Determine if the sentiment of `text` is positive or negative."""

# Create a simple sentiment analysis tool
result = sentiment_classifier(
    m, text="IBM is a great company for innovation and generative AI!"
)

# Display the result
print(f"Sentiment: {result}")

## Composing Summarizers with decision aides

Let's start controlling the program logic using LLM's.

Here we also expand the docstring of each of the function to the fullest extent.

In [ ]:
################################################################################
# COMPOSITIONALITY CHECKS                                                      #
################################################################################


@generative
def has_structured_conclusion(summary: str) -> Literal["yes", "no"]:
    """
    Determine whether the given summary contains a clearly marked conclusion or recommendation.

    This function analyzes the text structure to identify if there is an explicit
    conclusion section or clear recommendations. It looks for standard conclusive
    phrases, summary statements, or recommendation blocks.

    Args:
        summary (str): The text summary to analyze for structured conclusions.

    Returns:
        Literal["yes", "no"]: Returns "yes" if a structured conclusion is found,
                             "no" otherwise.
    """


@generative
def contains_actionable_risks(summary: str) -> Literal["yes", "no"]:
    """
    Analyze whether a summary contains actionable business risks or exposures.

    This function scans the text for mentions of business risks, threats, or
    vulnerabilities that require action. It looks for specific risk indicators
    and determines if they are presented in an actionable context.

    Args:
        summary (str): The text summary to analyze for actionable risks.

    Returns:
        Literal["yes", "no"]: Returns "yes" if actionable risks are identified,
                             "no" otherwise.
    """


@generative
def has_theme_and_plot(summary: str) -> Literal["yes", "no"]:
    """
    Analyze whether a summary contains both plot elements and thematic content.

    This function examines the text to determine if it includes both narrative
    elements (plot, story, events) and broader thematic elements (messages,
    meanings, interpretations).

    Args:
        summary (str): The text summary to analyze for plot and themes.

    Returns:
        Literal["yes", "no"]: Returns "yes" if both plot and thematic elements
                             are present, "no" otherwise.
    """

In [ ]:
################################################################################
# APPLY DECISION AIDES                                                         #
################################################################################

# Generate risk mitigation strategies based upon the meeting summary.
if contains_actionable_risks(m, summary=summary) == "yes":
    mitigation = generate_risk_mitigation(m, summary=summary)
    display(Markdown(f"## Mitigation:\n{mitigation}"))
else:
    display(Markdown("Summary does not contain actionable risks."))

In [ ]:
# Propose a business decision based upon the meeting summary.
if has_structured_conclusion(m, summary=summary) == "yes":
    decision = propose_business_decision(m, summary=summary)
    display(Markdown(f"# Decision:\n{decision}"))
else:
    display(Markdown("Summary lacks a structured conclusion."))

# Exercises

1. Define your own decision aides and compositionality checks and for a scenario specific to you.